In [1]:

import boto3
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import io
import tempfile
import os
from IPython.display import Image

s3 = boto3.resource('s3')
bucket_name = 'chemocraft-data'
folder_path = 'Data/BraTS20_Training_369 copy/'
bucket = s3.Bucket(bucket_name)

visualizer_folder = 'brain_visualizations'

def render_nii_from_s3(filename):
    print(f"Fetching file: {filename}")

    obj = bucket.Object(folder_path + filename)
    file_stream = io.BytesIO(obj.get()['Body'].read())

    with tempfile.NamedTemporaryFile(suffix='.nii', delete=False) as temp_file:  # Disable auto-delete
        temp_file.write(file_stream.getvalue())
        temp_file.flush()

        temp_file_path = temp_file.name
        print(f"Temporary file created: {temp_file_path}")

    try:
        img = nib.load(temp_file_path)
        data = img.get_fdata()

        print(f"Data shape for {filename}: {data.shape}")

        if data.size == 0:
            print(f"No data found in {filename}")
            return

        #slice_idx = data.shape[2] // 2 

        slice_idx = 88

        fig, ax = plt.subplots(figsize=(6, 6))
        imgdisplay = ax.imshow(data[:, :, slice_idx])
        fig.suptitle(f'{filename} - Slice {slice_idx}')
        plt.axis('off')  # Hide axes for cleaner display

        def update(frame):
            if frame<0 or frame>=data.shape[2]:
                return imgdisplay

            imgdisplay.set_data(data[:, :, frame])
            imgdisplay.figure.suptitle(f'{filename} - Slice {frame}')
            return imgdisplay

        ani = animation.FuncAnimation(fig=fig, func=update, frames=155, interval=50)
        print(f"Saving {filename}.gif")
        vid_path = os.path.join(visualizer_folder, f"video_{filename.split(".")[0]}.gif");
        ani.save(vid_path, writer="imagemagick")

    except Exception as e:
        print(f"Error loading file {filename}: {e}")
    finally:
        try:
            os.remove(temp_file_path)  # Clean up the temp file
            print(f"Deleted temporary file: {temp_file_path}")
        except OSError as cleanup_error:
            print(f"Error deleting temp file: {cleanup_error}")

found_files = False
for obj in bucket.objects.filter(Prefix=folder_path):
    if obj.key.endswith('.nii'):
        found_files = True
        filename = obj.key.split('/')[-1]  # Get the filename
        render_nii_from_s3(filename)

if not found_files:
    print(f"No .nii files found in the folder {folder_path}")




Hello, World
